In [1]:
from mcmc import *
from fit import *
from process_showers import ProcessEvents
from config import CounterConfig
# from datafiles import *
import matplotlib.pyplot as plt
plt.ion()
from utils import plot_event, plot_generator, get_data_files, preceding_noise_file


data_date_and_time = '20190504034237'
data_files = get_data_files(data_date_and_time)
noise_files = [preceding_noise_file(f) for f in data_files]
cfg = CounterConfig(data_files, noise_files)
# ckv = GetCkv(cfg)

In [2]:
# pars = [600.,2.e6,np.deg2rad(40.),np.deg2rad(315.), 450., -660.,-29,0,70,80.]
# ev = BasicParams.get_event(pars)
# pe = ProcessEvents(cfg, frozen_noise=False)
# real_nfits = pe.gen_nfits_from_event(ev)

In [3]:
corsika_file = '/home/isaac/NIZ/angle_cut_no_thinning/iact_DAT000064.dat'
pe = ProcessEvents(cfg, frozen_noise=True)
real_nfits = pe.gen_nfits_from_ei(corsika_file)

In [4]:
pf = NichePlane(real_nfits)
ty = tyro(real_nfits)

In [5]:
pf.theta

0.6971752406370161

In [6]:
import CHASM as ch
ei = ch.EventioWrapper(corsika_file)
xmax = ei.X[ei.nch.argmax()]
nmax = ei.nch.max()
theta = ei.theta
phi = ei.phi
pars = [xmax,nmax,theta,phi,437., -660.,-29.0,0,70,0.]
print(pars)

[525.0, 1200354.0, 0.7038913, 4.981169732409068, 437.0, -660.0, -29.0, 0, 70, 0.0]


In [7]:
guess = make_guess(ty,pf,cfg)
pardict = {p.name:pars[i] for i,p in enumerate(guess[:-1])}
pardict['t_offset'] = 64.

In [8]:
pt = PeakTimes(real_nfits, BasicParams, cfg)
pt.target_parameters = ['zenith','azimuth']
m = init_minuit(pt, guess)
m.tol = .1
m.simplex()

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 27.02 (χ²/ndof = 3.0)      │              Nfcn = 46               │
│ EDM = 0.0923 (Goal: 0.1)         │           time = 15.4 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    500    │    50     │            │            │   400   │   800   │  yes  │
│ 1 │ nmax     │   1.0e6   │   0.1e6   │            │            │  10000  │  1e+08  │  yes  │
│ 2 │ zenith   │ 705.74e-3 │  0.15e-3  │            │            │    0    │0.797175 │       │
│ 3 │ azimuth  │  4.98278  │  0.00020  │            │            │ 4.87478 │ 5.07478 │       │
│ 4 │ corex    │    436    │     5     │            │            │  379.5  │  489.2  │  yes  │
│ 5 │ corey    │   -664    │     5     │            │            │ -711.4  │ -615.1  │  yes  │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │     0     │    10     │            │            │  -450   │   450   │  yes  │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

Now with a good estimate for the angles, we can fit the pulse widths to estimate xmax.

In [9]:
guess = update_guess_values(guess, m)
pw = PulseWidth(real_nfits, BasicParams, cfg)
pw.target_parameters = ['xmax']
m = init_minuit(pw, guess)
m.simplex(ncall=40)

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 3.036 (χ²/ndof = 0.3)      │               Nfcn = 7               │
│ EDM = 0.018 (Goal: 0.1)          │            time = 2.9 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    490    │    90     │            │            │   400   │   800   │       │
│ 1 │ nmax     │   1.0e6   │   0.1e6   │            │            │  10000  │  1e+08  │  yes  │
│ 2 │ zenith   │   0.706   │   0.017   │            │            │    0    │0.797175 │  yes  │
│ 3 │ azimuth  │   4.983   │   0.017   │            │            │ 4.87478 │ 5.07478 │  yes  │
│ 4 │ corex    │    436    │     5     │            │            │  379.5  │  489.2  │  yes  │
│ 5 │ corey    │   -664    │     5     │            │            │ -711.4  │ -615.1  │  yes  │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │     0     │    10     │            │            │  -450   │   450   │  yes  │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

Now, with a good estimate for log(xmax), we can fit the integrated pulse areas for nmax.

In [10]:
guess = update_guess_values(guess, m)
pa = PulseArea(real_nfits, BasicParams, cfg)
pa.target_parameters = ['nmax']
m = init_minuit(pa, guess)
m.simplex(ncall=20)

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 10.6 (χ²/ndof = 1.1)       │              Nfcn = 15               │
│ EDM = 0.00691 (Goal: 0.1)        │            time = 6.1 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    490    │    50     │            │            │   400   │   800   │  yes  │
│ 1 │ nmax     │  1.064e6  │  0.019e6  │            │            │  10000  │  1e+08  │       │
│ 2 │ zenith   │   0.706   │   0.017   │            │            │    0    │0.797175 │  yes  │
│ 3 │ azimuth  │   4.983   │   0.017   │            │            │ 4.87478 │ 5.07478 │  yes  │
│ 4 │ corex    │    436    │     5     │            │            │  379.5  │  489.2  │  yes  │
│ 5 │ corey    │   -664    │     5     │            │            │ -711.4  │ -615.1  │  yes  │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │     0     │    10     │            │            │  -450   │   450   │  yes  │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

Now, with these values in the ballpark, we can simultaneously minimize both the shower profile and core position by fitting the normalized pulse areas.

In [11]:
guess = update_guess_values(guess, m)
pa = NormalizedPulseArea(real_nfits, BasicParams, cfg)
pa.target_parameters = ['xmax','nmax','corex','corey']
m = init_minuit(pa, guess)
m.simplex()

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 9.351 (χ²/ndof = 1.3)      │              Nfcn = 45               │
│ EDM = 0.055 (Goal: 0.1)          │           time = 18.2 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    478    │     9     │            │            │   400   │   800   │       │
│ 1 │ nmax     │  1.11e6   │  0.04e6   │            │            │  10000  │  1e+08  │       │
│ 2 │ zenith   │   0.706   │   0.017   │            │            │    0    │0.797175 │  yes  │
│ 3 │ azimuth  │   4.983   │   0.017   │            │            │ 4.87478 │ 5.07478 │  yes  │
│ 4 │ corex    │    438    │     5     │            │            │  379.5  │  489.2  │       │
│ 5 │ corey    │   -664    │    10     │            │            │ -711.4  │ -615.1  │       │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │     0     │    10     │            │            │  -450   │   450   │  yes  │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

In [12]:
guess = update_guess(m)
at = AllTunka(real_nfits, BasicParams, cfg)
at.target_parameters = ['t_offset']
m = init_minuit(at, guess)
m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 53.64 (χ²/ndof = 1.2)      │              Nfcn = 63               │
│ EDM = 5.8e-05 (Goal: 0.0002)     │           time = 25.9 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │    478    │     9     │            │            │   400   │   800   │  yes  │
│ 1 │ nmax     │  1.11e6   │  0.04e6   │            │            │  10000  │  1e+08  │  yes  │
│ 2 │ zenith   │   0.706   │   0.017   │            │            │    0    │0.797175 │  yes  │
│ 3 │ azimuth  │   4.983   │   0.017   │            │            │ 4.87478 │ 5.07478 │  yes  │
│ 4 │ corex    │    438    │     5     │            │            │  379.5  │  489.2  │  yes  │
│ 5 │ corey    │   -664    │    10     │            │            │ -711.4  │ -615.1  │  yes  │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │  85.4573  │  0.0007   │            │            │  -450   │   450   │       │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────┬───────────────────────────────────────────────────────────────────────────────────────────┐
│          │     xmax     nmax   zenith  azimuth    corex    corey    corez       x0   lambda t_offset │
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│     xmax │        0        0        0        0        0        0        0        0        0        0 │
│     nmax │        0        0        0        0        0        0        0        0        0        0 │
│   zenith │        0        0        0        0        0        0        0        0        0        0 │
│  azimuth │        0        0        0        0        0        0        0        0        0        0 │
│    corex │        0        0        0        0        0        0        0        0        0        0 │
│    corey │        0        0        0        0        0        0        0        0        0        0 │
│    corez │        0        0        0        0        0        0        0        0        0        0 │
│       x0 │        0        0        0        0        0        0        0        0        0        0 │
│   lambda │        0        0        0        0        0        0        0        0        0        0 │
│ t_offset │        0        0        0        0        0        0        0        0        0  4.4e-07 │
└──────────┴───────────────────────────────────────────────────────────────────────────────────────────┘

In [13]:
guess = update_guess(m)
at = AllSamples(real_nfits, BasicParams, cfg)
at.target_parameters = ['t_offset']
m = init_minuit(at, guess)


In [14]:
m.tol=.1
m.fixed = True
m.fixed['xmax'] = False
m.fixed['nmax'] = False
m.fixed['zenith'] = False
m.fixed['azimuth'] = False
m.fixed['corex'] = False
m.fixed['corey'] = False
m.fixed['t_offset'] = False
# m.scan()
m.simplex()

┌─────────────────────────────────────────────────────────────────────────┐
│                               Simplex                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 79.26 (χ²/ndof = 1.3)      │              Nfcn = 186              │
│ EDM = 0.155 (Goal: 0.1)          │           time = 64.6 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Hesse not run           │            NO covariance             │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name     │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ xmax     │   469.8   │    1.2    │            │            │   400   │   800   │       │
│ 1 │ nmax     │  1.12e6   │  0.01e6   │            │            │  10000  │  1e+08  │       │
│ 2 │ zenith   │ 703.66e-3 │  0.26e-3  │            │            │    0    │0.797175 │       │
│ 3 │ azimuth  │  4.98054  │  0.00017  │            │            │ 4.87478 │ 5.07478 │       │
│ 4 │ corex    │  438.70   │   0.18    │            │            │  379.5  │  489.2  │       │
│ 5 │ corey    │ -664.118  │   0.034   │            │            │ -711.4  │ -615.1  │       │
│ 6 │ corez    │    -29    │     1     │            │            │         │         │  yes  │
│ 7 │ x0       │     0     │     1     │            │            │         │         │  yes  │
│ 8 │ lambda   │    70     │     1     │            │            │   60    │   80    │  yes  │
│ 9 │ t_offset │ 85.45795  │  0.00012  │            │            │  -450   │   450   │       │
└───┴──────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

In [15]:
guess = update_guess_values(guess, m)
guessdict = {p.name:p for p in guess}
# guessdict['xmax'].limits = (guessdict['xmax'].value - 50., guessdict['xmax'].value + 50.)
# guessdict['nmax'].limits = (guessdict['nmax'].value - 2.e5, guessdict['nmax'].value + 2.e5)
# guessdict['zenith'].limits = (guessdict['zenith'].value - 5.e-3, guessdict['zenith'].value + 5.e-3)
# guessdict['azimuth'].limits = (guessdict['azimuth'].value - 5.e-3, guessdict['azimuth'].value + 5.e-3)
# guessdict['corex'].limits = (guessdict['corex'].value - 5., guessdict['corex'].value + 5.)
# guessdict['corey'].limits = (guessdict['corey'].value - 5., guessdict['corey'].value + 5.)
# guessdict['zenith'].fixed = True
# guessdict['azimuth'].fixed = True
# guessdict['corex'].fixed = True
# guessdict['corey'].fixed = True
guessdict['corez'].fixed = True
guessdict['x0'].fixed = True
guessdict['lambda'].fixed = True
# guessdict['t_offset'].fixed = True

In [16]:
initial_guessdict = {p.name:p.value for p in guess}
initial_guessdict['t_offset'] = 80.

In [17]:
names = [p.name for p in guess if not p.fixed]
ndim = len(names)

In [18]:
sampler, pos, prob, state = main(at,guess,niter=5000,nwalkers=ndim*2)

 27%|█████████▎                         | 1334/5000 [1:02:27<2:51:39,  2.81s/it]

emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:

Process ForkPoolWorker-5:


emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:

KeyboardInterrupt: 

In [ ]:
flat_samples = sampler.get_chain(discard=4000,flat=True)

samples = sampler.flatchain
best_sample = samples[np.argmax(sampler.flatlnprobability)]

fig = corner.corner(flat_samples,labels=names,quantiles=[0.16, 0.5, 0.84])

axes = np.array(fig.axes).reshape((ndim, ndim))

# Loop over the diagonal
for i,name in enumerate(names):
    ax = axes[i, i]
    ax.axvline(pardict[name], color="g", label='thrown')
    ax.axvline(initial_guessdict[name], color="b", label='initial guess')
    ax.axvline(best_sample[i], color="r", label = 'max likelihood')
    if i == 0:
        fig.legend()
    # ax.set_xlim(np.min([pardict[names[i]],initial_guessdict[names[i]],best_sample[i]]),np.max([pardict[names[i]],initial_guessdict[names[i]],best_sample[i]]))

    # Loop over the histograms
for yi in range(ndim):
    for xi in range(yi):
        ax = axes[yi, xi]
        # ax.set_xlim(np.min([pardict[names[xi]],initial_guessdict[names[xi]],best_sample[xi]]),np.max([pardict[names[xi]],initial_guessdict[names[xi]],best_sample[xi]]))
        # ax.set_ylim(np.min([pardict[names[yi]],initial_guessdict[names[yi]],best_sample[yi]]),np.max([pardict[names[yi]],initial_guessdict[names[yi]],best_sample[yi]]))
        ax.axvline(pardict[names[xi]], color="g")
        ax.axvline(best_sample[xi], color="r")
        ax.axhline(pardict[names[yi]], color="g")
        ax.axhline(best_sample[yi], color="r")
        ax.axvline(initial_guessdict[names[xi]], color="b")
        ax.axhline(initial_guessdict[names[yi]], color="b")
        ax.plot(pardict[names[xi]], pardict[names[yi]], "sg")
        ax.plot(best_sample[xi], best_sample[yi], "sr")
        ax.plot(initial_guessdict[names[xi]], initial_guessdict[names[yi]], "sb")

In [ ]:
pars

In [ ]:
guessdict = {p.name:p.value for p in guess}

In [ ]:
np.array(list(guessdict.values()))

In [ ]:
len(flat_samples)

In [ ]:
best_sample

In [ ]:
len(real_nfits)